In [1]:
import pandas as pd
import psycopg2
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import json


In [2]:
def consultar_datos(filename, db_name, table_name):
    try:
        with open(filename, 'r') as file:
            config = json.load(file)

        connection = psycopg2.connect(
            host=config["host"],
            user=config["user"],
            password=config["password"],
            dbname=db_name
        )

        query = f"SELECT * FROM {table_name}"
        water = pd.read_sql(query, connection)

        return water
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error al consultar datos: {error}")
        return None
    finally:
        if connection:
            connection.close()

if __name__ == "__main__":
    filename = 'db_config.json'
    db_name = 'db_water'
    table_name = 'water_table'
    water = consultar_datos(filename, db_name, table_name)

    if water is not None:
        print(water.head())


C:\Users\PAO\AppData\Local\Temp\ipykernel_24080\82420538.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  water = pd.read_sql(query, connection)


          Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni  \
0  2010-01-01            Bolívar        13        El Guamo      13248   
1  2010-01-01            Bolívar        13        El Guamo      13248   
2  2010-01-01            Bolívar        13        El Guamo      13248   
3  2010-01-01            Bolívar        13        El Guamo      13248   
4  2010-01-01            Bolívar        13        El Guamo      13248   

   IrcaMinimo  IrcaMaximo  IrcaPromedio NombreParametroAnalisis2  \
0         0.0       100.0         37.32        Alcanilidad Total   
1         0.0       100.0         37.32                 Aluminio   
2         0.0       100.0         37.32                 Arsénico   
3         0.0       100.0         37.32                   Cadmio   
4         0.0       100.0         37.32                   Calcio   

   MuestrasEvaluadas  NumeroParametrosMinimo  NumeroParametrosMaximo  \
0                 67                       2                       7   
1       

In [72]:
query = "SELECT * FROM table_water LIMIT 10"
first_lines = pd.read_sql(query, con=engine)

print(first_lines)

    Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni IrcaMinimo2  \
0  2010            Bolívar        13        El Guamo      13248           0   
1  2010            Bolívar        13        El Guamo      13248           0   
2  2010            Bolívar        13        El Guamo      13248           0   
3  2010            Bolívar        13        El Guamo      13248           0   
4  2010            Bolívar        13        El Guamo      13248           0   
5  2010            Bolívar        13        El Guamo      13248           0   
6  2010            Bolívar        13        El Guamo      13248           0   
7  2010            Bolívar        13        El Guamo      13248           0   
8  2010            Bolívar        13        El Guamo      13248           0   
9  2010            Bolívar        13        El Guamo      13248           0   

  IrcaMaximo IrcaPromedio     NombreParametroAnalisis2  MuestrasEvaluadas  \
0        100        37,32            Alcanilidad Tota

Leeimos las 10 primeras líneas del archivo muestran que:
•	El archivo utiliza ; (punto y coma) como delimitador.
•	Las líneas siguientes contienen datos, pero notamos que hay campos vacíos representados por la ausencia de valores entre los delimitadores (por ejemplo, ;;;;), lo que puede haber causado el problema de lectura inicial.

In [ ]:
water.info()

In [ ]:
water.describe()

Nos asegurándonos  que las columnas categoricas estén formateados consistentemente:

In [ ]:
water['NombreDepartamento'] = water['NombreDepartamento'].str.title().str.strip()
water['NombreMunicipio'] = water['NombreMunicipio'].str.title().str.strip()

water.head()


Convertimos todos los nombres a un formato estándar  y eliminamos espacios al inicio y al final, y reduciremos los espacios múltiples a uno solo.

Analizando valores faltantes:

In [73]:
missing_values = water.isnull().sum()

missing_values = missing_values[missing_values > 0]

missing_values


ResultadoMinimo      63.935478
ResultadoMaximo      63.935478
ResultadoPromedio    63.935478
dtype: float64


Las columnas ResultadoMinimo, ResultadoMaximo y ResultadoPromedio tienen valores faltantes, con un total de 262,361 valores faltantes en cada una. Esto sugiere que hay muchas mediciones en las que estos resultados no se proporcionaron o no aplican.

In [ ]:
missing_percentage = water[['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio']].isnull().mean() * 100

print(missing_percentage)



 realizamos un análisis de la distribución de los valores existentes en ResultadoMinimo, ResultadoMaximo, y ResultadoPromedio para entender mejor estos datos:

In [74]:
descriptive_stats = water[['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio']].describe()

descriptive_stats

,ResultadoMinimo,ResultadoMaximo,ResultadoPromedio
count,130973,130973,130973
unique,3642,8251,16909
top,0,0,0
freq,50007,13483,14498


La distribución de los valores en las columnas ResultadoMinimo, ResultadoMaximo, y ResultadoPromedio muestra una amplia gama de valores, desde negativos hasta positivos muy grandes, lo que indica una variabilidad significativa en los datos

Realizaremos un análisis para ver si hay alguna correlación entre las filas con datos faltantes y otras variables. Esto nos ayudaría a entender si los datos faltantes están relacionados con otras características de los datos:

In [75]:
water['missing_values'] = water[['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio']].isnull().any(axis=1)

correlation_with_missing = water.corr()['missing_values'].sort_values()

correlation_with_missing = correlation_with_missing.drop('missing_values')

plt.figure(figsize=(10, 8))
correlation_with_missing.plot(kind='bar')
plt.title('Correlación de la presencia de valores faltantes con otras columnas')
plt.xlabel('Columnas')
plt.ylabel('Correlación con valores faltantes')
plt.show()

NumeroMuestras             -0.233502
NumeroParametrosMaximo     -0.231124
MuestrasNoAptas            -0.199938
NumeroParametrosPromedio   -0.164568
NumeroParametrosMinimo     -0.067111
MuestrasTratadas           -0.033818
MuestrasEvaluadas          -0.031457
MuestrasSinTratar          -0.001355
Divi_muni                   0.012727
Div_dpto                    0.013504
Año                         0.071342
Name: missing_values, dtype: float64


Los resultados de la correlación muestran que no hay ninguna correlación fuerte con la presencia de valores faltantes, ya que los valores son relativamente bajos. Esto sugiere que los valores faltantes pueden no estar sistemáticamente relacionados con otras variables numéricas en nuestro dataset.

¿Existe una manera de imputar estos valores basada en otros datos disponibles?

Imputación por promedio:

In [ ]:
promedio_resultado_minimo = water['ResultadoMinimo'].mean()
promedio_resultado_maximo = water['ResultadoMaximo'].mean()
promedio_resultado_promedio = water['ResultadoPromedio'].mean()

water['ResultadoMinimo'].fillna(promedio_resultado_minimo, inplace=True)
water['ResultadoMaximo'].fillna(promedio_resultado_maximo, inplace=True)
water['ResultadoPromedio'].fillna(promedio_resultado_promedio, inplace=True)

water[['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio']].head(15)

Al utilizar el promedio, todos los valores faltantes se reemplazan por un único valor promedio, ignorando la variabilidad esencial entre diferentes muestras y localidades. Esto resulta en una representación simplificada que puede distorsionar el análisis, especialmente en estudios donde las diferencias en los niveles de contaminación son críticas para determinar la salud pública y la seguridad ambiental. La uniformidad impuesta por el promedio elimina la capacidad de detectar variaciones importantes en la calidad del agua, llevando potencialmente a interpretaciones erróneas y decisiones inadecuadas basadas en un entendimiento incompleto de los datos.

Imputación por mediana:

In [ ]:
mediana_resultado_minimo = water['ResultadoMinimo'].median()
mediana_resultado_maximo = water['ResultadoMaximo'].median()
mediana_resultado_promedio = water['ResultadoPromedio'].median()

water['ResultadoMinimo'].fillna(mediana_resultado_minimo, inplace=True)
water['ResultadoMaximo'].fillna(mediana_resultado_maximo, inplace=True)
water['ResultadoPromedio'].fillna(mediana_resultado_promedio, inplace=True)

water[['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio']].head()

Al igual que en el caso de la imputación por promedio, la mediana asigna un único valor a todos los registros faltantes de una columna, ignorando la variabilidad y las relaciones entre variables

En el contexto de nuestro trabajo, que implica analizar la calidad del agua a través de los parámetros ResultadoMinimo, ResultadoMaximo, y ResultadoPromedio, enfrentamos desafíos significativos al imputar los valores faltantes con métodos generales como el promedio o la mediana. Estos enfoques no capturan la complejidad y la variabilidad específica de los datos relacionados con la calidad del agua, donde cada parámetro puede variar considerablemente por factores geográficos, temporales y ambientales.

Por lo tanto, en este caso específico, la eliminación de las columnas ResultadoMinimo, ResultadoMaximo, y ResultadoPromedio puede ser la decisión más prudente. Esta acción elimina el riesgo de introducir sesgos significativos a través de la imputación y nos permite enfocarnos en partes del conjunto de datos con información completa y precisa

In [ ]:
water = water.drop(['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio'], axis=1)

Volvemos a revisar que no se presenten mas valores faltantes:

In [ ]:
water_faltantes = water.isnull().sum()


water_faltantes[water_faltantes > 0]

Investigación de filas duplicadas:

In [79]:
filas_duplicadas_antes = water.duplicated().sum()

water = water.drop_duplicates()

filas_duplicadas_despues = water.duplicated().sum()

filas_duplicadas_antes, filas_duplicadas_despues

(0, 0)

No se encontraron filas duplicadas en el dataset

 Verificación de los tipos de datos de las columnas para asegurar que coincidan con el contenido que albergan.

In [81]:
tipos_datos_actuales = water.dtypes

tipos_datos_actuales

Año                          int64
NombreDepartamento          object
Div_dpto                     int64
NombreMunicipio             object
Divi_muni                    int64
IrcaMinimo2                 object
IrcaMaximo                  object
IrcaPromedio                object
NombreParametroAnalisis2    object
MuestrasEvaluadas            int64
MuestrasTratadas             int64
MuestrasSinTratar            int64
NumeroParametrosMinimo       int64
NumeroParametrosMaximo       int64
NumeroParametrosPromedio     int64
NumeroMuestras               int64
MuestrasNoAptas              int64
PorcentajeNoAptas           object
Codigo                      object
missing_values                bool
dtype: object

Cambio tipo de dato de "Año":

In [65]:
water['Año'] = pd.to_datetime(water['Año'], format='%Y')

water.dtypes

Año                         datetime64[ns]
NombreDepartamento                  object
Div_dpto                             int64
NombreMunicipio                     object
Divi_muni                            int64
IrcaMinimo2                         object
IrcaMaximo                          object
IrcaPromedio                        object
NombreParametroAnalisis2            object
MuestrasEvaluadas                    int64
MuestrasTratadas                     int64
MuestrasSinTratar                    int64
NumeroParametrosMinimo               int64
NumeroParametrosMaximo               int64
NumeroParametrosPromedio             int64
NumeroMuestras                       int64
MuestrasNoAptas                      int64
PorcentajeNoAptas                   object
Codigo                              object
missing_values                        bool
dtype: object

In [66]:
%pip install unidecode
import pandas as pd
from unidecode import unidecode

for col in water.select_dtypes(include=['object']).columns:
    water[col] = water[col].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

water.head()

,Año,NombreDepartamento,NombreMunicipio,IrcaMinimo2,IrcaMaximo,IrcaPromedio,NombreParametroAnalisis2,MuestrasEvaluadas,MuestrasTratadas,MuestrasSinTratar,NumeroParametrosMinimo,NumeroParametrosMaximo,NumeroParametrosPromedio,NumeroMuestras,MuestrasNoAptas
0,2010-01-01,Bolívar,El Guamo,0,100,"37,32",Alcanilidad Total,67,67,0,2,7,2,1,0
1,2010-01-01,Bolívar,El Guamo,0,100,"37,32",Aluminio,67,67,0,2,7,2,0,0
2,2010-01-01,Bolívar,El Guamo,0,100,"37,32",Arsénico,67,67,0,2,7,2,0,0
3,2010-01-01,Bolívar,El Guamo,0,100,"37,32",Cadmio,67,67,0,2,7,2,0,0
4,2010-01-01,Bolívar,El Guamo,0,100,"37,32",Calcio,67,67,0,2,7,2,1,0


In [ ]:
water = water.drop(columns=[ 'missing_values'])


water.head()

Valores atipicos 

In [ ]:

import pandas as pd

outliers_count = {}


numeric_cols = water.select_dtypes(include=['float64', 'int64']).columns

Q1 = water[numeric_cols].quantile(0.25)
Q3 = water[numeric_cols].quantile(0.75)
IQR = Q3 - Q1


limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR


outliers_count = ((water[numeric_cols] < limite_inferior) | (water[numeric_cols] > limite_superior)).sum()

print("Número de valores atípicos por columna:")
print(outliers_count)

Iniciaremos la imputación de valores atipicos con nuestra columna de mayor importancia "IRCA PROMEDIO"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 6))


bp = plt.boxplot(water['IrcaPromedio'], vert=False, patch_artist=True, showmeans=True,
                 boxprops=dict(facecolor='lightblue', color='blue'),
                 whiskerprops=dict(color='blue'),
                 capprops=dict(color='blue'),
                 medianprops=dict(color='red'),
                 meanprops=dict(marker='D', markeredgecolor='black', markerfacecolor='yellow'))


plt.title('Distribución del IRCA Promedio', fontsize=16)
plt.xlabel('IRCA Promedio', fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
plt.tight_layout()


median_value = np.median(water['IrcaPromedio'])
plt.annotate(f'Mediana: {median_value:.2f}', xy=(median_value, 0.5), xytext=(median_value, 0.6),
             arrowprops=dict(facecolor='black', arrowstyle="->"),
             horizontalalignment='center', verticalalignment='top')


plt.show()

que vemos en el grafico?

In [ ]:
Q1 = water['IrcaPromedio'].quantile(0.25)
Q3 = water['IrcaPromedio'].quantile(0.75)
IQR = Q3 - Q1


limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

valores_atipicos = water[(water['IrcaPromedio'] < limite_inferior) | (water['IrcaPromedio'] > limite_superior)]

muestras_atipicas_detalles = valores_atipicos[['Año', 'NombreDepartamento', 'NombreMunicipio', 'IrcaPromedio']]

print(muestras_atipicas_detalles.head())

In [ ]:
water['EsAtipico'] = (water['IrcaPromedio'] < limite_inferior) | (water['IrcaPromedio'] > limite_superior)

valores_atipicos_por_departamento = water.groupby('NombreDepartamento')['EsAtipico'].sum()

valores_atipicos_por_departamento = valores_atipicos_por_departamento.sort_values(ascending=False)

print(valores_atipicos_por_departamento)

In [ ]:
def obtener_valores_atipicos_por_departamento(df, departamento, limite_inferior, limite_superior):

    datos_departamento = df[df['NombreDepartamento'] == departamento]

    
    valores_atipicos = datos_departamento[
        (datos_departamento['IrcaPromedio'] < limite_inferior) |
        (datos_departamento['IrcaPromedio'] > limite_superior)
    ]

    return valores_atipicos

nombre_departamento = 'Magdalena' #Cambia el deartamento que desees observar
valores_atipicos_departamento = obtener_valores_atipicos_por_departamento(water, nombre_departamento, limite_inferior, limite_superior)


print(valores_atipicos_departamento[['NombreMunicipio', 'IrcaPromedio']])


explicacion de por que los vamos a dejar los atipicos

Valor atipicos de IRCA minimo

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(water['IrcaMinimo2'], kde=True, bins=30)
plt.title('Distribución de IRCA Mínimo')
plt.xlabel('IRCA Mínimo')
plt.ylabel('Frecuencia')
plt.show()

La distribución de IrcaMinimo muestra una concentración de valores en un rango específico, con la presencia de una cola hacia valores más altos. Esta distribución sugiere que, aunque la mayoría de los valores son relativamente bajos (indicando un riesgo bajo a moderado de la calidad del agua), hay una cantidad significativa de valores que se extienden hacia rangos más altos, lo cual podría ser indicativo de situaciones de riesgo más elevado en ciertas áreas.

Conclusión atipicos de IRCA: Estos resultados son consistentes con el contexto práctico de los datos, donde valores más altos de IRCA minimo y promedio, indican un mayor riesgo asociado con la calidad del agua La presencia de estos valores extremos, lejos de ser errores o anomalías sin significado práctico, subraya la importancia de monitorear y mejorar la calidad del agua en áreas donde el IRCA es alto. Estos análisis resaltan la variabilidad y los desafíos en la gestión de la calidad del agua a nivel local y regional.

Creando analisis más detallado sobre la calidad del agua en nuestros datos:


Identificamos los parámetros de análisis que tienen mayor influencia en la contaminación del agua:

In [ ]:
parametros_influencia = water.groupby('NombreParametroAnalisis2')['IrcaPromedio'].mean().sort_values(ascending=False)

top_20_parametros = parametros_influencia.head(15)

top_20_parametros

Todos estos parámetros tienen un valor promedio de "IrcaPromedio" de aproximadamente 25, lo que sugiere una influencia significativa en la calidad del agua según este índice

In [ ]:
water = water[water['NombreParametroAnalisis2'].isin(top_20_parametros.index)]

water.head(), water.shape

Hemos filtrado el dataset para dejar solo las filas correspondientes a los top 20 parámetros de análisis relacionados con la contaminación del agua y los demas han salido eliminados

Transformación de columnas:

In [ ]:
def clasificar_irca(irca):
    if irca == 0:
        return 'Sin información'
    elif 0.001 <= irca <= 5:
        return 'Sin riesgo'
    elif 5.001 <= irca <= 14:
        return 'Bajo'
    elif 19.001 <= irca <= 35:
        return 'Medio'
    elif 54.001 <= irca <= 70:
        return 'Alto'
    elif 124.001 <= irca <= 100:
        return 'Inviable'
    else:
        return 'No clasificado' 

water['rango_irca'] = water['IrcaPromedio'].apply(clasificar_irca)


In [ ]:
water['porcentaje_muestras_tratadas'] = (water['MuestrasTratadas'] / water['MuestrasEvaluadas']) * 100

water[[ 'MuestrasEvaluadas', 'MuestrasTratadas', 'porcentaje_muestras_tratadas']].head()


 "Porcentaje de Muestras Tratadas" fue generada a partir del dataset original, utilizando las columnas de Muestras Tratadas y Muestras Evaluadas para calcular el porcentaje de agua que ha sido sometida a tratamiento en diversas localidades. 

In [ ]:
water['diferencia_muestras_tratadas_sin_tratar'] = abs(water['MuestrasTratadas'] - water['MuestrasSinTratar'])

water[['MuestrasTratadas', 'MuestrasSinTratar', 'diferencia_muestras_tratadas_sin_tratar']].head()

La columna "Diferencia entre Muestras Tratadas y Sin Tratar" fue desarrollada para cuantificar la brecha en el número de muestras de agua que han sido sometidas a tratamiento en comparación con aquellas que no lo han sido dentro de cada conjunto de datos evaluados

In [ ]:
water['rango_parametros_analizados'] = water['NumeroParametrosMaximo'] - water['NumeroParametrosMinimo']

water[[ 'NumeroParametrosMinimo', 'NumeroParametrosMaximo', 'rango_parametros_analizados']].head()

La columna "Rango de Parámetros Analizados" fue creada para proporcionar una medida de la variabilidad en el análisis de calidad del agua, reflejando la diferencia entre el número máximo y mínimo de parámetros químicos y biológicos examinados en cada muestra.

In [ ]:
columnas_a_eliminar = ['MuestrasTratadas', 'MuestrasEvaluadas', 'MuestrasSinTratar',
                      'NumeroParametrosMinimo', 'NumeroParametrosMaximo']
water = water.drop(columns=columnas_a_eliminar)


water.head()

Exportamos el nuevo archivo con el procesamiento y transformaciones:

In [ ]:
water.to_csv('water_cleaned.csv', index=False)